In [1]:
from astropy.io import fits
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
cal_dir = Path(os.environ['DATA'], 'DDS', 'Taka', 'Calibration')
str(cal_dir)

def fitsread(f, header=False):
    with fits.open(f) as hdul:
        data = hdul[0].data
        if header:
            h = hdul[0].header
            return data, h
        else:
            return data
        
def rescale(image, mref, sref, roi=None):
    imageV = image if roi is None else image[roi]
    m1 = np.median(imageV)
    s1 = mad(imageV)
    ki = sref/s1
    rimage = ki*(image - m1) + mref
    return rimage


def loopSigmaClip(images, rej_mask):
    imquad2 = np.ma.masked_array(images, mask=rej_mask.copy())
    medquad2 = np.ma.median(imquad2, axis=0)
        
    sigma = imquad2.std(axis=0)
    rej_mask = (np.abs(imquad2 - medquad2) > 5*sigma)
    return rej_mask


def sigmaClip(images):
    rejMask = np.zeros(images.shape, dtype=np.bool)
    n = 1
    while n > 0:
        rejMask0 = rejMask.copy()
        rejMask = loopSigmaClip(images, rejMask)
        n = rejMask.sum()
        print(n)
        rejMask = rejMask0 | rejMask.filled(False)

    return rejMask


def loopWinSigmaClipping(images, med, sigma, rejMask, qmask=None):
    
    m0 = med - 1.5*sigma
    m1 = med + 1.5*sigma
    mask_min = images < m0
    mask_max = images > m1
    images[mask_min] = np.tile(m0, (images.shape[0],1,1))[mask_min]
    images[mask_max] = np.tile(m1, (images.shape[0],1,1))[mask_max]
    sigma0 = sigma.copy()
    
    images = np.ma.masked_array(images.copy(), mask=rejMask.copy())
    med = np.ma.median(images, axis=0).filled(0)
        
    # Apply pixel rejection mask before calculating new sigma
    if qmask is not None:
        sigma[qmask] = 1.134*images.std(axis=0)[qmask]
    else:
        sigma = 1.134*images.std(axis=0)
    qmask = np.abs(sigma - sigma0)/sigma0 > 0.0005
    return qmask, sigma, med


def winsorizedSigmaClipping(images, verbose=False):
    
    rejMask = np.zeros(images.shape, dtype=np.bool)
    n = 1
    while n > 0:
        print('starting outer loop')
        images_ma = np.ma.masked_array(images.copy(), mask=rejMask)
        m = np.ma.median(images_ma, axis=0).filled()
        sigma = images_ma.std(axis=0).filled()
        t = images.copy()

        n1 = 1
        qmask = None
        print('starting inner loop')
        while n1>0:
            qmask, sigma, m = loopWinSigmaClipping(t, m, sigma, rejMask, qmask=qmask)
            n1 = qmask.sum()
            if verbose: print(f'loop n1={n1}')
        print('finished winsorized loop')
        rejMask0 = rejMask.copy()
        rejMask = np.abs(images_ma - m) > 5*sigma 
        n = rejMask.sum()
        print('n = ', n)
        rejMask = rejMask0 | rejMask.filled(False)
        
    return rejMask

# Bias

In [4]:
bias_dir = Path(cal_dir, 'Bias_bin1_Mar_2018')
biasf = list(bias_dir.rglob('Bias*.fit'))
print(len(biasf))
d0, h0 = fitsread(biasf[0], header=True)
nx = h0['NAXIS1']
ny = h0['NAXIS2']
nx2 = int(nx/2)
ny2 = int(ny/2)
print(nx,ny)
print(nx2, ny2)

200
4524 3624
2262 1812


In [ ]:
q1 = np.s_[0:ny2, 0:nx2]
q2 = np.s_[0:ny2, nx2:]
q3 = np.s_[ny2:, 0:nx2]
q4 = np.s_[ny2:, nx2:]

In [ ]:
mbias_mean = np.zeros([ny, nx])
mbias_median = np.zeros([ny, nx])
mbias_sc = np.zeros([ny, nx])
mbias_wsc = np.zeros([ny, nx])

# q1 = np.s_[0:512, 0:512]
imquad = np.zeros([len(biasf), ny2, nx2])
for quad in [q1, q2, q3, q4]:
# for quad in [q1,]:
    for i, f in enumerate(biasf):
            im = fitsread(f)
            imquad[i] = im[quad]
            
#     # Artithmetic mean
#     mbias_mean[quad] = np.float32(imquad.mean(axis=0))
#     # Median
#     mbias_median[quad] = np.float32(np.median(imquad, axis=0))
#     # Sigma Clipping
#     rejMask = sigmaClip(imquad)
#     print('rejected pixels (sigma clipping) n=', rejMask.sum())  
#     imquad_ma = np.ma.masked_array(imquad, mask=rejMask)
#     mbias_sc[quad] = np.float32(imquad_ma.mean(axis=0).filled(0))
    # Winsorized Sigma Clipping
    rejMask = winsorizedSigmaClipping(imquad, verbose=True)
    print('rejected pixels (winsorized sigma clipping) n=', rejMask.sum())    
    imquad_ma = np.ma.masked_array(imquad, mask=rejMask)
    mbias_wsc[quad] = np.float32(imquad_ma.mean(axis=0).filled(0))
    

# fits.writeto(Path(cal_dir, 'mBias_bin1_mean_Mars_2018.fits'), mbias_mean, overwrite=True)
# fits.writeto(Path(cal_dir, 'mBias_bin1_median_Mars_2018.fits'), mbias_median, overwrite=True)
# fits.writeto(Path(cal_dir, 'mBias_bin1_sc_Mars_2018.fits'), mbias_sc, overwrite=True)
fits.writeto(Path(cal_dir, 'mBias_bin1_wsc_Mars_2018.fits'), mbias_wsc, overwrite=True)

In [ ]:
q1 = np.s_[0:1024, 0:1024]
imquad = np.zeros([len(biasf[0:100]), 1024, 1024])

for i, f in enumerate(biasf[0:50]):
        im = fitsread(f)
        imquad[i] = im[q1]

In [ ]:
def sigmaClip2(images):
    rejMask = np.zeros(images.shape, dtype=np.bool)
    n = 1
    while n > 0:
        rejMask0 = rejMask.copy()
        med = np.nanmedian(images, axis=0)
        sigma = np.nanstd(images, axis=0)
        rejMask = (np.abs(images - med) > 5*sigma)
        n = rejMask.sum()
#         print(n)
        rejMask = rejMask0 | rejMask
        images[rejMask] = np.nan

    return rejMask

In [ ]:
%%time
# Sigma Clipping
rejMask = sigmaClip(imquad.copy())
# print('rejected pixels n=', rejMask.sum())  
# imquad_ma = np.ma.masked_array(imquad, mask=rejMask)
# mBiasQ = imquad_ma.mean(axis=0).filled(0)

In [ ]:
%%time
rejMask1 = sigmaClip2(imquad.copy())

In [ ]:
imquad2 = imquad.copy()
rejMask2 = np.zeros([100, 1024, 1024], dtype=np.bool)

In [ ]:
%%timeit
imquad2 = imquad.copy()
for r in range(1024):
    rejMask2[:,r,:] = sigmaClip2(imquad2[:,r,:])

In [ ]:
%%timeit
imquad2 = imquad.copy()
rejMask = np.array([sigmaClip2(imquad2[:,r,:]) for r in range(1024)])

In [ ]:
imquad3 = np.zeros([1024, 1024, len(biasf[0:100])])

for i, f in enumerate(biasf[0:50]):
        im = fitsread(f)
        imquad3[..., i] = im[q1]

In [ ]:
def sigmaClip3(images):
    rejMask = np.zeros(images.shape, dtype=np.bool)
    n_outliers = []
    n = 1
    while n > 0:
        rejMask0 = rejMask.copy()
        med = np.nanmedian(images, axis=-1)
        sigma = np.nanstd(images, axis=-1)
        rejMask = (np.abs(images - med[...,np.newaxis]) > 5*sigma[...,np.newaxis])
        n = rejMask.sum()
        n_outliers.append(n)
#         print(n)
        rejMask = rejMask0 | rejMask
        images[rejMask] = np.nan

    return rejMask, n_outliers

In [ ]:
%%time
rejMask3, nout = sigmaClip3(imquad3.copy())

In [ ]:
print(nout)

In [ ]:
d = sigmaClip3(imquad4[1,:,:])

In [ ]:
imquad4 = imquad3.copy()
rejMask4, nout = np.array([sigmaClip3(imquad4[r,:,:]) for r in range(1024)])

In [ ]:
%%timeit
imquad4 = imquad3.copy()
rejMask4 = np.array([sigmaClip3(imquad4[r,:,:]) for r in range(1024)])

In [ ]:
%%time
for r in range(1024):
    rejMask4[r,:,:] = sigmaClip3(imquad4[r,:,:])

In [ ]:
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 16

vmin, vmax = np.percentile(avgquad, 0.01), np.percentile(avgquad, 99)
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20,10))
axs[0].imshow(avgquad, vmin=vmin, vmax=vmax, cmap='gray')
axs[1].imshow(mBiasQ, vmin=vmin, vmax=vmax, cmap='gray')
#axs[0].axis('off'), axs[1].axis('off')
plt.tight_layout()
axs[0].set_title('Master Bias: Arithmetic mean')
axs[0].text(50, 280, f'min = {avgquad.min():.2f} \nmax = {avgquad.max():.2f} \nmean = {avgquad.mean():.2f} \nsigma = {avgquad.max():.2f}', color='black',
           bbox=dict(facecolor='none', edgecolor='black', boxstyle='round'))
axs[1].set_title(f'Master Bias: Sigma clipping - pixels rejected = {rejMask.sum()}')
axs[1].text(50, 280, f'min = {mBiasQ.min():.2f} \nmax = {mBiasQ.max():.2f} \nmean = {mBiasQ.mean():.2f} \nsigma = {mBiasQ.max():.2f}', color='black',
           bbox=dict(facecolor='none', edgecolor='black', boxstyle='round'))
print(avgquad.min(), mBiasQ.min(), avgquad.max(), mBiasQ.max(), avgquad.std(), mBiasQ.std())

In [ ]:
fits.writeto(Path(cal_dir, 'masterBias_bin1_average_Mars_2018.fits'), mbias_bin1.astype(np.float32), overwrite=True)
del imquad
print(mbias_bin1.shape)

In [ ]:
mbias_bin1 = fitsread(Path(cal_dir, 'masterBias_bin1_average_Mars_2018.fits'))

In [ ]:
bias_dir = Path(cal_dir, 'Bias_bin2_Nov_2018')
biasf = list(bias_dir.rglob('Bias*.fit'))
print(len(biasf))
biases = np.array([fitsread(f) for f in biasf])
mbias_bin2 = biases.mean(axis=0)

In [ ]:
fits.writeto(Path(cal_dir, 'masterBias_bin2_average_Nov_2018.fits'), mbias_bin2.astype(np.float32), overwrite=True)
del biases
print(mbias_bin2.shape)

# Darks

In [ ]:
dark_dir = Path(cal_dir, 'Darks_Jan_2019')
darkf600 = list(dark_dir.rglob('*600s*.fit'))
darkf300 = list(dark_dir.rglob('*300s*.fit'))
len(darkf600), len(darkf300)

In [ ]:
darks600 = np.array([fitsread(f) for f in darkf600])
mdark600 =darks600.mean(axis=0)

In [ ]:
fits.writeto(Path(cal_dir, 'masterDark600_bin1_average_Jan_2019.fits'), mdark600.astype(np.float32))

In [ ]:
darks300 = np.array([fitsread(f) for f in darkf300])
mdark300 = darks300.mean(axis=0)

In [ ]:
fits.writeto(Path(cal_dir, 'masterDark300_bin2_average_Jan_2019.fits'), mdark300.astype(np.float32))

# Flats

In [ ]:
flats_dir = Path(cal_dir, 'Flats')
hflatsf = list(flats_dir.rglob('Hflat_Mar_3_2019/HFlat*.fit'))

rflatsf = list(flats_dir.rglob('RFlat_Jan_26_2019/RFlat*.fit'))
gflatsf = list(flats_dir.rglob('GFlat_Feb_3_2019/GFlat*.fit'))
bflatsf = list(flats_dir.rglob('BFlat_Feb_4_2019/BFlat*.fit'))
rgbflatsf = [rflatsf, gflatsf, bflatsf]
len(rflatsf), len(gflatsf), len(bflatsf), len(hflatsf)

In [ ]:
mdark600_flat = mdark600-mbias_bin1

with fits.open(hflatsf[0]) as hdul:
    nx = hdul[0].header['NAXIS1']
    ny = hdul[0].header['NAXIS2']
    exposure = hdul[0].header['EXPTIME']

exp_scale1 = exposure/600

masterHFlat = np.zeros([len(hflatsf), ny, nx])
for i, f in enumerate(hflatsf):
    with fits.open(f) as hdul:
        masterHFlat[i] = hdul[0].data -exp_scale1*mdark600_flat

m0 = np.median(masterHFlat[0])
masterHFlat = np.array([m0/np.median(flat)*flat for flat in masterHFlat])
masterHFlat = masterHFlat.mean(axis=0)

In [ ]:
fits.writeto(Path(cal_dir, 'masterHFlat_bin1_Mar_3_2019.fits'), masterHFlat, overwrite=True)

In [ ]:
mdark300_flat = mdark300-mbias_bin2

masterRGBFlats = []
for xflatsf in rgbflatsf:
    
    with fits.open(xflatsf[0]) as hdul:
        nx = hdul[0].header['NAXIS1']
        ny = hdul[0].header['NAXIS2']
        exposure = hdul[0].header['EXPTIME']

    exp_scale1 = exposure/300
    masterXFlat = np.zeros([len(xflatsf), ny, nx])
    for i, f in enumerate(xflatsf):
        with fits.open(f) as hdul:
            masterXFlat[i] = hdul[0].data - exp_scale1*mdark300_flat
    
    m0 = np.median(masterXFlat[0])
    masterXFlat = np.array([m0/np.median(flat)*flat for flat in masterXFlat])
    
    masterXFlat = masterXFlat.mean(axis=0)
    masterRGBFlats.append(masterXFlat)

In [ ]:
fits.writeto(Path(cal_dir, 'masterRFlat_bin2_Jan_26_2019.fits'), masterRGBFlats[0], overwrite=True)
fits.writeto(Path(cal_dir, 'masterGFlat_bin2_Feb_3_2019.fits'), masterRGBFlats[1], overwrite=True)
fits.writeto(Path(cal_dir, 'masterBFlat_bin2_Feb_4_2019.fits'), masterRGBFlats[2], overwrite=True)